In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

In [0]:
# If executing in Google Colab
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pickle.load(open("/content/drive/My Drive/IR/new_data/vocab_train.pkl", "rb"))

gc.collect()

15

In [0]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200
0,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,...,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0
1,0.009468,-0.000207,0.006507,0.002932,0.003958,0.000497,0.003386,-0.002031,0.011167,-0.000164,0.006493,0.001676,0.002899,0.001014,0.003889,-0.004367,0.003076,-0.003625,0.006377,0.002020,0.002293,0.001946,0.001255,0.001337,0.001696,0.000522,0.002819,0.009670,0.002090,0.005686,0.003602,-0.003354,-0.002510,-0.010026,-0.000831,0.002339,0.003184,-0.000239,0.001216,-0.000687,...,-0.000671,-0.010457,-0.000003,0.005779,0.010296,0.010935,-0.002979,-0.004037,0.005992,-0.012678,0.005986,-0.006107,-0.005234,-0.007335,-0.009172,0.007000,-0.004930,-0.000124,-0.002717,0.005612,-0.004379,0.015416,-0.009679,0.003662,-0.024018,-0.001816,0.006162,0.002506,0.004768,-0.003402,-0.001173,0.000589,0.007575,0.004242,0.019260,-0.032599,-0.004474,-0.000381,-0.013922,0
2,0.014384,0.007151,0.008332,0.006091,0.004625,0.004529,0.004504,-0.003184,0.012453,0.000592,0.016003,0.004294,0.005507,-0.002172,0.031945,-0.013505,0.000984,-0.002591,0.004048,-0.001282,0.001532,0.003503,0.009647,0.010772,0.001372,-0.004442,0.007900,0.011235,0.005642,0.000655,0.008783,-0.008866,-0.003140,-0.010913,0.000894,0.000116,0.001194,0.003508,0.032112,0.003696,...,0.000118,-0.005981,-0.007821,-0.000540,0.000701,-0.010183,0.001481,0.007101,0.002785,0.000756,-0.008660,-0.002016,-0.003169,-0.010294,-0.031118,-0.012604,0.003757,0.009463,0.001690,-0.002739,0.002081,0.006411,-0.008499,0.001247,-0.008603,-0.003093,-0.004554,-0.002134,0.002894,-0.003201,-0.003671,-0.003066,-0.001019,0.001080,0.000284,-0.004668,0.002035,-0.000705,0.002105,0
3,0.005805,-0.000445,0.006277,0.002650,0.001180,0.000530,0.003471,-0.000861,0.007929,-0.000036,0.008717,0.002888,0.004485,0.000102,0.020856,-0.004127,-0.000016,-0.005350,0.009743,-0.008355,0.002720,0.000007,0.004401,0.000242,0.002047,-0.001055,0.003525,0.014837,-0.002456,-0.002618,0.004014,-0.004979,-0.008325,0.001355,0.003423,0.002689,0.001622,-0.008582,0.004227,0.000907,...,0.000447,-0.009263,-0.011583,-0.005684,-0.000217,-0.007602,0.003265,0.010793,0.003550,0.000090,-0.010383,-0.001368,-0.000164,-0.013902,-0.041398,-0.020459,0.003787,0.015085,0.001520,-0.004766,0.003523,0.008579,-0.012007,0.000536,-0.011705,-0.002395,-0.004324,-0.001177,0.002535,0.000715,-0.003599,0.000186,0.000129,0.001767,0.001148,-0.003742,-0.000193,0.000054,0.002202,0
4,0.036517,0.000796,0.028990,0.020047,0.013289,0.016044,0.028873,-0.028521,0.042650,-0.007035,0.098507,0.075786,-0.016393,-0.013318,0.315229,-0.181502,0.067643,0.063728,-0.234932,0.087165,0.019009,-0.021675,-0.041784,-0.088812,-0.016162,0.016513,0.007038,0.022226,0.001718,-0.004419,0.018351,-0.023042,-0.043457,-0.053349,-0.034080,0.039774,-0.019924,-0.084057,-0.009131,0.028102,...,-0.004194,0.016235,0.012613,0.068632,-0.007173,0.050547,-0.012315,-0.006583,0.016042,-0.004047,0.101370,-0.003388,0.022891,0.186539,0.405127,0.400295,-0.206149,-0.182573,0.011915,0.051730,-0.060764,0.060493,0.034304,0.003559,-0.039798,0.024318,0.016669,0.007058,-0.003399,-0.006989,0.045200,0.001473,0.015099,-0.001031,0.011530,0.019863,

In [0]:
X_pred = pd.read_pickle("/content/drive/My Drive/IR/new_data/vocab_gs_test.pkl")#).drop(columns = ['doc_id'])


In [0]:
X_pred.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
0,0.582729,-0.022832,0.029596,0.104525,-0.343118,-0.217401,-0.043557,-0.139249,-0.072029,-0.097685,0.012233,-0.074913,-0.091606,0.008156,-0.044778,0.005641,0.084178,-0.042614,-0.034784,0.011972,-0.129292,-0.023437,-0.085000,0.094984,-0.108678,-0.016990,-0.164359,-0.155846,-0.117829,0.034375,0.009397,-0.103388,-0.022958,0.015397,0.035791,-0.059172,-0.107262,-0.001244,-0.008718,0.071589,...,0.003670,0.001241,0.000702,0.006749,-0.003041,0.001363,0.000420,-0.002128,-0.002107,-0.000684,-0.002380,-0.001825,0.000292,-0.000671,0.001825,0.003963,0.002842,0.004010,0.004184,-0.000100,0.006427,-0.002220,-0.004023,-0.006423,-0.005496,-0.002281,0.007076,0.000934,0.003217,-0.001312,-0.002597,-0.001761,-0.003476,-0.005861,0.000026,0.002485,0.005622,-0.005833,-0.006047,-0.001274
1,0.401628,0.185626,0.018010,0.106602,-0.288283,-0.201296,-0.092194,-0.161138,-0.087318,-0.033559,0.005561,-0.119432,0.011547,0.057281,-0.028681,0.003682,-0.028362,-0.057733,0.043131,0.009752,-0.007504,-0.032746,-0.036242,0.025518,-0.042211,0.193761,-0.271516,0.075757,0.520664,-0.108302,-0.052660,-0.188326,0.013321,0.004898,-0.032462,0.116719,0.033139,0.064590,-0.131452,0.047988,...,0.007355,-0.007424,-0.004142,-0.006023,-0.052062,-0.021643,-0.021411,0.004743,0.026734,0.001815,-0.004760,-0.015892,0.007179,0.006563,-0.007123,-0.003863,0.003925,-0.001901,-0.008789,0.003028,-0.006564,0.000032,0.006083,-0.001124,0.005187,-0.005969,-0.008102,0.000450,0.006005,0.002741,-0.005722,-0.002301,0.006388,-0.005237,-0.000484,0.000240,0.008475,-0.010218,0.002607,-0.003335
2,0.103828,0.217057,-0.298169,0.228082,-0.090615,-0.219259,-0.007757,-0.173342,-0.027032,-0.016815,0.014140,-0.065763,-0.002282,0.027318,0.032483,-0.026457,-0.019699,-0.034859,0.004158,0.022732,-0.026619,-0.064111,-0.001198,0.002654,0.014706,0.021423,-0.043366,-0.003069,0.077234,0.041017,-0.028756,0.038767,0.026344,0.010182,0.008830,0.006905,-0.074478,-0.066620,-0.074166,-0.053163,...,0.010168,-0.006747,-0.017212,-0.002664,-0.001715,0.003013,0.002580,0.019505,0.006682,-0.001811,-0.007143,-0.010624,-0.011168,-0.018720,0.013468,0.018697,-0.002287,-0.004924,-0.001154,-0.010389,0.008293,-0.001961,0.003523,0.001010,-0.009757,-0.009362,-0.004007,0.006718,0.023423,0.018636,0.005075,0.000344,-0.006365,0.007534,-0.006162,-0.013458,-0.011498,-0.009541,-0.011215,0.026463
3,0.216821,0.368665,0.470634,-0.074847,-0.035396,-0.195079,-0.158463,-0.037732,-0.112888,-0.022091,-0.013186,-0.024321,-0.034575,-0.005728,0.018967,-0.024676,-0.018155,-0.020058,0.011051,0.029749,-0.002558,-0.037219,-0.032048,0.013823,-0.004720,0.035886,-0.096136,0.033949,0.187413,0.027618,0.007230,-0.062142,0.048660,-0.041068,-0.014701,0.082915,-0.040374,-0.066615,-0.129794,-0.000989,...,0.016044,-0.005304,-0.012367,0.015246,-0.050245,-0.014292,-0.015452,0.005563,0.013156,-0.015550,-0.005442,-0.002433,0.001501,-0.002154,0.025648,0.025972,0.020215,0.050665,0.017487,-0.001344,0.007923,-0.030634,0.057305,0.020305,0.003854,-0.002602,0.021844,-0.000245,-0.048999,0.058494,-0.000654,0.000511,0.001090,0.123530,0.037037,0.000830,-0.058898,-0.034015,-0.029024,-0.031621
4,0.179181,0.382166,-0.551837,0.410862,-0.176760,-0.413063,-0.020254,-0.313282,-0.061645,-0.029644,0.006735,-0.115392,-0.064172,0.031642,0.006062,-0.020642,-0.008982,-0.066577,0.022535,0.005212,-0.079394,-0.029021,0.020283,0.002132,0.026187,0.033491,0.025617,-0.014687,-0.011306,-0.047788,-0.007859,0.009672,0.004805,0.004009,-0.002178,0.001821,0.005351,-0.016841,0.002023,-0.008748,...,0.000112,0.006790,-0.004002,-0.008931,0.000399,0.001893,0.001188,0.002686,0.004352,0.000292,0.000498,-0.002791,0.000927,0.000180,0.000953,0.002605,0.003202,-0.003826,-0.005782,0.003441,0.001317,-0.000177,-0.003026,0.001308,0.001033,-0.001117,-0.001592,-0.

In [0]:
data.shape

(613890, 201)

In [0]:
X_pred.shape

(8916, 200)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score


In [0]:
target = data.iloc[:,-1] 
data2 = data.iloc[:,:-1]

#data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data



In [0]:
data2.shape

(613890, 200)

In [0]:
new_data, X_test, new_target, y_test = train_test_split(data2, target, test_size = 0.95, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [0]:
new_data.shape

(30694, 200)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(new_data, new_target, test_size = 0.50, random_state = 32) # in this our main data is split into train and test

gc.collect()

0

In [0]:
X_train.shape

(15347, 200)

In [0]:
from sklearn.model_selection import RandomizedSearchCV
import time
from scipy.stats import uniform


model = SVC()

# Create regularization penalty space
kernel = ['rbf', 'poly','linear']

# Create regularization hyperparameter distribution using uniform distribution
C = [20,30, 40, 100, 150, 200,300,500]

# Create hyperparameter options
hyperparameters = dict(C=C, kernel = kernel)

# Create randomized search 5-fold cross validation and 100 iterations
clf = RandomizedSearchCV(model, hyperparameters, random_state=42, n_iter=24, cv=2, verbose=1, n_jobs=-1, scoring = 'f1')

s = time.time()
best_model = clf.fit(X_train, y_train)
e = time.time()

# View best hyperparameters
# print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
# print('Best C:', best_model.best_estimator_.get_params()['C'])
# print("Model Operation Completed in :",(e-s)/60,"min")

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 10.4min finished


In [0]:
 best_model.best_estimator_

SVC(C=500, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
pred = best_model.predict(X_test)
print(f1_score(pred, y_test))

0.23511450381679386


In [0]:
#pred = best_model.predict(X_test)
print(accuracy_score(pred, y_test))

0.8041311005408223


In [0]:
model_svm = SVC()

model_svm.fit(X_train, y_train)
print(model_svm.score(X_test, y_test))

In [0]:
pred = model_svm.predict(X_test)
print(f1_score(pred, y_test))

In [0]:
#import pickle


In [0]:
Pkl_Filename = "/content/drive/My Drive/IR/Pickle_svm_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model_svm, file)

In [0]:
# # Load the Model back from file
# with open(Pkl_Filename, 'rb') as file:  
#     Pickled_LR_Model = pickle.load(file)

# Pickled_LR_Model